Création d'une nouvelle base d'entraînement et d'évaluation à partir de la base de donnée LibriSpeech : 

On prend aléatoirement un nombre de speakers entre 2 et 4, on prend pour chacun des speakers un nombre d'intervention aléatoire (2 à 4). Chaque intervention est une portion aléatoire du speech initiale. On recole les blocs de façon aléatoire pour avoir un nouvel extrait. 

On réécrit à la fin des nouveaux fichiers wav.

In [ ]:
import glob
import os
import pandas as pd
import numpy as np
import shutil
import librosa
from tqdm import tqdm
from scipy.io.wavfile import write

In [ ]:
import ast

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
path = "/content/drive/MyDrive/hackaton twelve/AudioMNIST/LibriSpeech/LibriSpeech"

In [ ]:
df2 = pd.DataFrame(columns = ["file_id","freq","sampling_rate"])


for fichier in os.listdir(path):
  if '.flac'==fichier[-5:]:
        res = librosa.load(os.path.join(path, fichier))
        freq, sampling_rate = res[0],res[1]
        line_df = pd.DataFrame([[fichier, freq, sampling_rate]], columns = df2.columns)
        df2 = pd.concat([df2, line_df], ignore_index = True)
        

In [ ]:
df2['ID'] = df2["file_id"].apply(lambda x: x.split('-')[0])
df2["ID"] = df2["ID"].astype('int64')

In [ ]:
label = pd.read_csv(path + "/label",sep=',')
label = label.drop(["Unnamed: 0"],axis =1)
label.columns = ['ID','SEX']

In [ ]:
df2 = df2.merge(label, left_on='ID', right_on='ID')

In [ ]:
import random

In [78]:
df3 = pd.DataFrame(columns = ["freq","nb_pers","nb_f","nb_h", "timing"])

for i in range(200):
  nb_speaker = random.randint(2, 4)
  nb_blocs = [random.randint(1,3) for i in range(nb_speaker)]

  liste_decoup = []
  for i in range(nb_speaker):
    decoup = []
    for j in range(nb_blocs[i]):
      decoup.append(np.random.uniform(0,1))
      liste_decoup.append(sorted(decoup))
  
  df_sample = df2.sample(n=nb_speaker)
  df_sample_freq = df_sample["freq"].values
  df_sample_sex = df_sample["SEX"].values


  liste_morceaux = []

  for k in range(nb_speaker):
    sex = df_sample_sex[k]
    i = liste_decoup[k]
    if len(i) == 1:
      n = round(i[0] * len(df_sample_freq[k]) )
      morceau = df_sample_freq[k][:n]

      liste_morceaux.append((np.array(morceau),k,sex))
   
    if len(i) == 2:
      n = round( i[0] * len(df_sample_freq[k]) )
      n1 = round( i[1] * len(df_sample_freq[k]) )
      morceau = df_sample_freq[k][:n]
      morceau1 = df_sample_freq[k][n+1:n1]
      liste_morceaux.append((np.array(morceau),k,sex))
      liste_morceaux.append((np.array(morceau1),k,sex))
    
    if len(i) == 3:
      n = round( i[0] * len(df_sample_freq[k]) )
      n1 = round( i[1] * len(df_sample_freq[k]) )
      n2 = round( i[2] * len(df_sample_freq[k]) )
      morceau = df_sample_freq[k][:n]
      morceau1 = df_sample_freq[k][n+1:n1]
      morceau2 = df_sample_freq[k][n1+1:n2]
      liste_morceaux.append((np.array(morceau),k,sex))
      liste_morceaux.append((np.array(morceau1),k,sex))
      liste_morceaux.append((np.array(morceau2),k,sex))

  random.shuffle(liste_morceaux)
  liste_morceaux2 = [x[0] for x in liste_morceaux]
  liste_N = [ len(i) for i in liste_morceaux2 ]
  liste_duree = [ x/22050 for x in liste_N ]
  
  res = []
  t = 0
  for p in range(len(liste_morceaux2)):
    t+= liste_duree[p]
    res.append((liste_morceaux[p][1],t,liste_morceaux[p][2]))
  
  liste_morceaux2 = np.concatenate(liste_morceaux2)
  # On compte le nombre de voix de femmes de d'hommes piochés 
  compteur_f = 0 
  compteur_h = 0 

  genres = df_sample["SEX"].values

  for i in genres:
    if i == ' M ':
      compteur_h += 1
    if i == ' F ':
      compteur_f += 1

  line_df = pd.DataFrame([[ liste_morceaux2, nb_speaker , compteur_f, compteur_h,res ]], columns = df3.columns)
  df3 = pd.concat([df3, line_df], ignore_index = True)

In [80]:
timings = []

for i in range(200):
  liste = df3['timing'].values[i]

  new_list = []
  (speak_id, start_time, sex) = liste.pop(0)

  while liste :
    new_speak_id, new_start_time, new_sex = liste.pop(0)

    if speak_id == new_speak_id :
      start_time += new_start_time

    else :
      new_list.append((speak_id, start_time, sex))
      speak_id, start_time, sex = new_speak_id, new_start_time, new_sex
  timings.append(new_list)


[(2, 1.4854421768707482, ' M '), (3, 3.51437641723356, ' M '), (2, 3.606530612244898, ' M '), (2, 4.698820861678005, ' M '), (1, 8.837006802721088, ' F '), (0, 12.391746031746031, ' M '), (1, 17.5815873015873, ' F ')]
[(2, 1.7310657596371881, ' M '), (0, 4.416780045351474, ' M '), (2, 9.608344671201813, ' M '), (1, 14.630521541950113, ' F '), (1, 21.401632653061224, ' F '), (2, 26.67002267573696, ' M ')]
[(3, 6.516281179138322, ' M '), (3, 9.225034013605443, ' M '), (0, 17.294285714285714, ' M '), (2, 17.834648526077096, ' F '), (1, 18.355419501133785, ' F '), (3, 18.76517006802721, ' M '), (2, 30.93034013605442, ' F ')]
[(2, 7.847709750566893, ' F '), (0, 15.700680272108844, ' M '), (1, 22.148117913832202, ' M '), (1, 27.271156462585036, ' M ')]
[(2, 6.035555555555556, ' M '), (3, 10.231791383219955, ' F '), (3, 12.558367346938775, ' F '), (1, 17.324081632653062, ' M '), (0, 27.21451247165533, ' M ')]
[(2, 0.6778684807256236, ' F '), (3, 2.5696598639455783, ' M '), (1, 5.7823582766439

In [104]:
timings2 = []

for i in timings:
  new_list = []
  start_time = 0
  (speaker, duration, sex) = i[0]
  new_list.append((speaker,0,duration,sex))
  for j in range(1, len(i)-1):
    duration2 = i[j+1][1] - i[j][1]
    new_start_time = start_time + duration 

    (speaker, start_time, duration, sex) = i[j][0], new_start_time, duration2, i[j][2]
    
    new_list.append((speaker, start_time, duration, sex))
  timings2.append(new_list)

timings2

[[(2, 0, 1.4854421768707482, ' M '),
  (3, 1.4854421768707482, 4.7909750566893425, ' M '),
  (2, 6.276417233560091, 0.5316553287981858, ' M '),
  (1, 6.8080725623582765, 3.5547392290249427, ' F ')],
 [(2, 0, 1.7310657596371881, ' M '),
  (0, 1.7310657596371881, 5.191564625850339, ' M '),
  (2, 6.922630385487527, 26.423809523809524, ' M ')],
 [(3, 0, 15.741315192743766, ' M '),
  (0, 15.741315192743766, 0.5403628117913826, ' M '),
  (2, 16.28167800453515, 0.5207709750566885, ' F '),
  (1, 16.802448979591837, 0.40975056689342537, ' F ')],
 [(2, 0, 7.847709750566893, ' F ')],
 [(2, 0, 6.035555555555556, ' M '),
  (3, 6.035555555555556, -5.466077097505668, ' F ')],
 [(2, 0, 0.6778684807256236, ' F '),
  (3, 0.6778684807256236, 3.2126984126984133, ' M ')],
 [(1, 0, 6.006349206349206, ' F '),
  (2, 6.006349206349206, 1.642993197278912, ' F '),
  (1, 7.649342403628118, 6.48952380952381, ' F '),
  (2, 14.138866213151928, 0.794603174603175, ' F '),
  (3, 14.933469387755103, 1.863492063492064, '

In [105]:
df3.timing = timings2

In [106]:
df3.to_csv('nouvelle_base_2.csv')

In [107]:
df3.head()

,freq,nb_pers,nb_f,nb_h,timing,len_timling
0,"[0.050662443, 0.05233018, 0.053912535, 0.05480...",4,1,3,"[(2, 0, 1.4854421768707482, M ), (3, 1.485442...",4
1,"[-0.0011938515, -0.007182852, -0.012882587, -0...",3,1,2,"[(2, 0, 1.7310657596371881, M ), (0, 1.731065...",3
2,"[-0.00023306884, -9.711788e-05, -0.00018447044...",4,2,2,"[(3, 0, 15.741315192743766, M ), (0, 15.74131...",4
3,"[-0.00083981245, -0.0009235549, -0.00045078128...",3,1,2,"[(2, 0, 7.847709750566893, F )]",1
4,"[-2.6354796e-06, -1.4959975e-06, 7.119993e-06,...",4,1,3,"[(2, 0, 6.035555555555556, M ), (3, 6.0355555...",2


In [108]:
from scipy.io.wavfile import write

data = df3["freq"][5]
samplerate = 22050

In [109]:
for i in range(200):
  data = df3['freq'][i]
  write(f'/content/drive/MyDrive/hackaton twelve/nouvelle base 2/new_{i}.wav', samplerate, data)

